# Description

TODO

# Modules loading

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from ccc import conf
from ccc.coef import ccc

# Settings

In [2]:
display(conf.GENERAL["N_JOBS"])

15

In [3]:
DATA_SIZES = [
    200,
    600,
    1800,
]

# split data size in this many points
K_MAX_N_SPLITS = 10

# always include this value since it is the default we use in CCC
DEFAULT_K_MAX = 10

# N_REPS = 10

In [4]:
np.random.seed(0)

# Paths

In [5]:
OUTPUT_DIR = conf.RESULTS_DIR / "k_max_test"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
display(OUTPUT_DIR)

PosixPath('/pl/active/pivlab/projects/ccc/base/results/k_max_test')

# Run

In [6]:
# initialize (i.e., compile with numba)
ccc(np.random.rand(100), np.random.rand(100))

0.02047038824894002

In [7]:
results = pd.DataFrame(columns=["data_size", "k_max", "k_max_as_n_fraction", "coef"])

idx = 0
for data_size in tqdm(DATA_SIZES):
    # get the values for k_max to try...
    k_max_splits = np.linspace(2, data_size, K_MAX_N_SPLITS)
    # ... but always add the default k_max used by CCC
    k_max_splits = [int(i) for i in np.sort(np.append(k_max_splits, DEFAULT_K_MAX))]

    # generate random data
    # TODO: if I generate normal data, what happens?
    # d1 = np.random.rand(data_size)
    # d2 = np.random.rand(data_size)
    d1 = np.random.normal(size=data_size)
    d2 = np.random.normal(size=data_size)

    for k_max in tqdm(k_max_splits):
        c = ccc(d1, d2, internal_n_clusters=k_max, n_jobs=conf.GENERAL["N_JOBS"])

        results.loc[idx] = [data_size, k_max, k_max / data_size, c]
        idx += 1

        # save
        results.to_pickle(OUTPUT_DIR / "k_max-results.pkl")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  9%|▉         | 1/11 [00:00<00:05,  1.78it/s]

 45%|████▌     | 5/11 [00:00<00:00,  9.26it/s]

 73%|███████▎  | 8/11 [00:01<00:00,  5.60it/s]

...

 91%|█████████ | 10/11 [1:13:39<16:51, 1011.31s/it]

100%|██████████| 11/11 [2:08:28<00:00, 1698.43s/it]

100%|██████████| 11/11 [2:08:28<00:00, 700.77s/it] 


100%|██████████| 3/3 [2:10:35<00:00, 3559.95s/it]

100%|██████████| 3/3 [2:10:35<00:00, 2611.75s/it]

# Check

In [8]:
results.shape

(33, 4)

In [9]:
assert results.shape[0] == int(len(DATA_SIZES) * (K_MAX_N_SPLITS + 1))

In [10]:
results.head()

,data_size,k_max,k_max_as_n_fraction,coef
0,200.0,2.0,0.01,0.043594
1,200.0,10.0,0.05,0.043594
2,200.0,24.0,0.12,0.043594
3,200.0,46.0,0.23,0.043594
4,200.0,68.0,0.34,0.043594
